# Part 1: load the LOC data and identify non-numeric IDs, storing them separately:

In [1]:
import pandas as pd
 
file_path = '/Users/twylazhang/Desktop/Econ_banks_Research/LOC.dta'

# Load the .dta file using pandas read_stata function
loc_df = pd.read_stata(file_path)

loc_df

,source,lender_TopRSSDID,lender_TopName
0,1.0,1003790,USE CREDIT UNION
1,1.0,100393,FRANKENMUTH CREDIT UNION
2,3.0,100571,DEPARTMENT OF LABOR FCU
3,1.0,1025608,FIRST HAWAIIAN INC
4,3.0,1026801,FREMONT BANCORPORATION
...,...,...,...
1074,1.0,f96580,EXCEL FEDERAL CREDIT UNION
1075,3.0,f968744,FIRST CHOICE BANK
1076,1.0,f97,CAP
1077,1.0,f98,CAPITAL CITY


In total, there are 1079 rows of data

In [2]:
# Rename 'lender_TopRSSDID' to 'id' for uniformity with other datasets
loc_df.rename(columns={'lender_TopRSSDID': 'id'}, inplace=True)

# Keep non-numeric IDs in a separate DataFrame called non_numeric_ids_df
non_numeric_ids_loc_df = loc_df[~loc_df['id'].str.isdigit()].copy()
numeric_ids_loc_df = loc_df[loc_df['id'].str.isdigit()].copy()

# Convert 'id' to integer now that non-numeric values are removed from numeric_ids_loc_df
numeric_ids_loc_df['id'] = numeric_ids_loc_df['id'].astype(int)

# Now numeric_ids_loc_df contains only numeric IDs, and non_numeric_ids_loc_df contains the non-numeric IDs


There are 666 rows in non_numeric_ids_df, and 413 rows in loc_df after seperate numberic and non-numeric data

In [3]:
#non_numeric_ids_loc_df.to_csv('non_numeric_lenders.csv', index=False)

In [4]:
non_numeric_ids_loc_df

,source,id,lender_TopName
413,2.0,f1,1/0 HOLDCO LLC
414,1.0,f10,ACRE HOLDINGS LLC
415,1.0,f100,CB LENDING
416,3.0,f101,CB&T SYNOVUS
417,1.0,f102,CBBC
...,...,...,...
1074,1.0,f96580,EXCEL FEDERAL CREDIT UNION
1075,3.0,f968744,FIRST CHOICE BANK
1076,1.0,f97,CAP
1077,1.0,f98,CAPITAL CITY


In [5]:
numeric_ids_loc_df

,source,id,lender_TopName
0,1.0,1003790,USE CREDIT UNION
1,1.0,100393,FRANKENMUTH CREDIT UNION
2,3.0,100571,DEPARTMENT OF LABOR FCU
3,1.0,1025608,FIRST HAWAIIAN INC
4,3.0,1026801,FREMONT BANCORPORATION
...,...,...,...
408,3.0,967699,MIDUSA CREDIT UNION
409,3.0,971986,REACH FEDERAL CREDIT UNION
410,3.0,978471,UNITED STATES SENATE FEDERAL CREDIT UNION
411,1.0,986177,ADVIA CREDIT UNION


# Part 2: Match numeric_ids_loc_df with call report and FR report

1. **Initial Matching with BHC Data**:
   - Match LOC IDs to BHC data to determine if the LOC entries correspond to BHCs.

2. **Identify Parent Banks and Subsidiaries in Call Report**:
   - **Parent Banks**: If the `id` column matches the `parent_id` column, the bank is identified as a **parent** bank.
   - **Subsidiaries**: If the `id` column does not match the `parent_id` column, the bank is identified as a **subsidiary**.

3. **Secondary Matching with Call Reports**:
   - For IDs not matched to BHCs:
     - Check against IDs of the **parent** bank in the call reports and tag them as 'Call_Parent'.
     - If still unmatched, check against IDs of **subsidiary** banks in the call reports and tag them as 'Call_Subsidiary'.

4. **Determine the Highest Parent ID for Subsidiaries** (only for banks identified as **Call_Subsidiary**):
   - **Identify Subsidiary IDs** based on the previous classification as `Call_Subsidiary`.
   - **Search in FR Report**: Check if the subsidiary ID exists in the FR report.
     - If found, use this as the highest parent ID.
     - If not found, proceed to the next step.
   - **Search in Call Report**:
     - For IDs not found in the FR report, trace their parent IDs in the call report.
     - Update the current ID to the found parent ID and use this found parent ID to recheck against the FR report.
     - Continue this process until the highest parent ID is identified.
   - **Add the highest parent ID** as a new column in the main DataFrame (`numeric_ids_loc_df`).

In [6]:
# Load the call report and FR report
call_df = pd.read_csv('/Users/twylazhang/Desktop/Econ_banks_Research/background/call.csv')
fr_report = pd.read_csv('/Users/twylazhang/Desktop/Econ_banks_Research/background/FR_Report.csv')

# Normalize ID columns in all datasets for consistent integer comparison
fr_report['id'] = pd.to_numeric(fr_report['id'], errors='coerce').fillna(0).astype(int)
call_df['id'] = pd.to_numeric(call_df['id'], errors='coerce').fillna(0).astype(int)
call_df['parent_id'] = pd.to_numeric(call_df['parent_id'], errors='coerce').fillna(0).astype(int)

In [7]:
# Step 1: Initial Matching with BHC Data
numeric_ids_loc_df['Match_Type'] = 'No Match'
numeric_ids_loc_df.loc[numeric_ids_loc_df['id'].isin(fr_report['id']), 'Match_Type'] = 'BHC'

# Step 2: Identify Parent Banks and Subsidiaries in Call Report
call_df['Entity_Type'] = 'Subsidiary'
call_df.loc[call_df['id'] == call_df['parent_id'], 'Entity_Type'] = 'Parent'

# Step 3: Secondary Matching with Call Reports for Parents and Subsidiaries
condition = numeric_ids_loc_df['Match_Type'] == 'No Match'
# Match with Parent Banks
numeric_ids_loc_df.loc[condition & numeric_ids_loc_df['id'].isin(call_df[call_df['Entity_Type'] == 'Parent']['id']), 'Match_Type'] = 'Call_Parent'
# Match with Subsidiaries
numeric_ids_loc_df.loc[condition & numeric_ids_loc_df['id'].isin(call_df[call_df['Entity_Type'] == 'Subsidiary']['id']), 'Match_Type'] = 'Call_Subsidiary'


In [8]:
numeric_ids_loc_df

,source,id,lender_TopName,Match_Type
0,1.0,1003790,USE CREDIT UNION,No Match
1,1.0,100393,FRANKENMUTH CREDIT UNION,No Match
2,3.0,100571,DEPARTMENT OF LABOR FCU,No Match
3,1.0,1025608,FIRST HAWAIIAN INC,BHC
4,3.0,1026801,FREMONT BANCORPORATION,BHC
...,...,...,...,...
408,3.0,967699,MIDUSA CREDIT UNION,No Match
409,3.0,971986,REACH FEDERAL CREDIT UNION,No Match
410,3.0,978471,UNITED STATES SENATE FEDERAL CREDIT UNION,No Match
411,1.0,986177,ADVIA CREDIT UNION,No Match


In [9]:
fr_report.columns

Index(['id', 'date', 'fed_district_code', 'charter_type', 'city', 'country',
       'parent_id', 'lei', 'name_legal', 'name', 'org_type', 'fed_regulator',
       'state'],
      dtype='object')

In [10]:
call_df

,date,id,charter_type,name_legal,name,org_type,fed_district_code,city,country,state,fed_regulator2,lei,parent_id,Entity_Type
0,20171231.0,37,200.0,BANK OF HANCOCK COUNTY,BANK OF HANCOCK CTY,1.0,6.0,SPARTA,UNITED STATES,GA,FDIC,0,37,Parent
1,20171231.0,242,200.0,FIRST COMMUNITY BANK XENIA-FLORA,FIRST CMNTY BK XENIA FLORA,1.0,8.0,XENIA,UNITED STATES,IL,FRS,0,3088643,Subsidiary
2,20171231.0,279,300.0,"MINEOLA COMMUNITY BANK, SSB",MINEOLA CMNTY BK SSB,6.0,11.0,MINEOLA,UNITED STATES,TX,FDIC,0,3619720,Subsidiary
3,20171231.0,354,200.0,BISON STATE BANK,BISON ST BK,1.0,10.0,BISON,UNITED STATES,KS,FDIC,0,354,Parent
4,20171231.0,457,200.0,LOWRY STATE BANK,LOWRY ST BK,1.0,9.0,LOWRY,UNITED STATES,MN,FDIC,0,1127016,Subsidiary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5957,20171231.0,5086072,200.0,BLUE GATE BANK,BLUE GATE BK,1.0,12.0,COSTA MESA,UNITED STATES,CA,FDIC,0,5086072,Parent
5958,20171231.0,5087752,0.0,HUDSON BRANCH,ROYAL BK OF CANADA HUDSON BR,0.0,0.0,JERSEY CITY,UNITED STATES,NJ,OCC,0,5087752,Parent
5959,20171231.0,5087949,400.0,"MB FINANCIAL INTERNATIONAL, INC.",MB FNCL INTL,1.0,4.0,ROSEMONT,UNITED STATES,IL,FRS,0,1090987,Subsidiary
5960,20171231.0,5113866,0.0,NEW YORK BRANCH,TAIWAN BUS BK NY BR,0.0,0.0,NEW YORK,UNITED STATES,NY,FRS,0,5113866,Parent


In [11]:
# Step 4: Find the Highest Parent ID for Call_Subsidiary Rows
def find_highest_parent_id(subsidiary_id, call_df, fr_report):
    current_id = subsidiary_id
    highest_parent_id = None

    # First, search in fr_report for the current ID
    if current_id in fr_report['parent_id'].values:
        highest_parent_id = current_id
        print(f"Subsidiary ID {subsidiary_id}: Found in Parent_ID column in FR report as {current_id}.")
    elif current_id in fr_report['id'].values:
        highest_parent_id = current_id
        print(f"Subsidiary ID {subsidiary_id}: Found in ID column in FR report as {current_id}.")
    else:
        print(f"Subsidiary ID {subsidiary_id}: Not found in FR report.")
        
        # If not found in fr_report, search in call_df for parent ID
        while True:
            parent_id_series = call_df.loc[call_df['id'] == current_id, 'parent_id']
            if not parent_id_series.empty:
                parent_id = parent_id_series.values[0]
                print(f"Subsidiary ID {subsidiary_id}: Found parent ID {parent_id} in Call report for current ID {current_id}.")
                current_id = parent_id
                if current_id in fr_report['parent_id'].values:
                    highest_parent_id = current_id
                    print(f"Subsidiary ID {subsidiary_id}: Parent ID {current_id} found in FR report after searching Call report.")
                    break
                elif current_id in fr_report['id'].values:
                    highest_parent_id = current_id
                    print(f"Subsidiary ID {subsidiary_id}: ID {current_id} found in FR report after searching Call report.")
                    break
                else:
                    print(f"Subsidiary ID {subsidiary_id}: Parent ID {current_id} not found in FR report. Continuing search in Call report.")
            else:
                print(f"Subsidiary ID {subsidiary_id}: Parent ID for current ID {current_id} not found in Call report.")
                break

    return highest_parent_id

# Initialize the highest_parent_id column
numeric_ids_loc_df['highest_parent_id'] = None

# Apply the search to each Call_Subsidiary
for idx, row in numeric_ids_loc_df[numeric_ids_loc_df['Match_Type'] == 'Call_Subsidiary'].iterrows():
    subsidiary_id = row['id']
    highest_parent_id = find_highest_parent_id(subsidiary_id, call_df, fr_report)
    numeric_ids_loc_df.at[idx, 'highest_parent_id'] = highest_parent_id


Subsidiary ID 2980209: Not found in FR report.
Subsidiary ID 2980209: Found parent ID 5006575 in Call report for current ID 2980209.
Subsidiary ID 2980209: Parent ID 5006575 found in FR report after searching Call report.
Subsidiary ID 3614958: Not found in FR report.
Subsidiary ID 3614958: Found parent ID 5116344 in Call report for current ID 3614958.
Subsidiary ID 3614958: ID 5116344 found in FR report after searching Call report.
Subsidiary ID 494261: Not found in FR report.
Subsidiary ID 494261: Found parent ID 2875332 in Call report for current ID 494261.
Subsidiary ID 494261: ID 2875332 found in FR report after searching Call report.


In [12]:
# Filter the DataFrame to find rows of Call_Subsidiary
non_null_highest_parent_id_df = numeric_ids_loc_df[numeric_ids_loc_df['highest_parent_id'].notna()]

non_null_highest_parent_id_df

,source,id,lender_TopName,Match_Type,highest_parent_id
195,1.0,2980209,BARCLAYS BANK DELAWARE,Call_Subsidiary,5006575
260,1.0,3614958,CALIFORNIA BANK OF COMMERCE,Call_Subsidiary,5116344
354,2.0,494261,PACIFIC WESTERN BANK,Call_Subsidiary,2875332


In [13]:
# Define the columns to display
columns_to_display = ['source', 'id', 'lender_TopName', 'Match_Type']

# Display rows that are classified as BHC
bhc_rows = numeric_ids_loc_df[numeric_ids_loc_df['Match_Type'] == 'BHC'][columns_to_display]
print("Rows classified as BHC:")
print(bhc_rows)
print("Number of BHC rows:", bhc_rows.shape[0])
print("\n")
# Display rows that are classified as Call_Parent
call_parent_rows = numeric_ids_loc_df[numeric_ids_loc_df['Match_Type'] == 'Call_Parent'][columns_to_display]
print("Rows classified as Call_Parent:")
print(call_parent_rows)
print("Number of Call_Parent rows:", call_parent_rows.shape[0])
print("\n")
# Display rows that are classified as Call_Subsidiary
call_subsidiary_rows = numeric_ids_loc_df[numeric_ids_loc_df['Match_Type'] == 'Call_Subsidiary'][['id', 'lender_TopName', 'Match_Type','highest_parent_id']]
print("Rows classified as Call_Subsidiary:")
print(call_subsidiary_rows)
print("Number of Call_Subsidiary rows:", call_subsidiary_rows.shape[0])

Rows classified as BHC:
     source       id                    lender_TopName Match_Type
3       1.0  1025608                FIRST HAWAIIAN INC        BHC
4       3.0  1026801            FREMONT BANCORPORATION        BHC
5       3.0  1027004                          ZIONS BC        BHC
6       1.0  1030040                 BANK OF CMRC HOLD        BHC
7       1.0  1030170                  TRICO BANCSHARES        BHC
..      ...      ...                               ...        ...
352     3.0  4876838                    PUGET SOUND BC        BHC
356     2.0  4973353             HARBORONE BANCORP INC        BHC
357     1.0  4980409                        OP BANCORP        BHC
358     1.0  4981648                        MARQUIS BC        BHC
359     1.0  4991076  SMITH & HOOD HOLDING COMPANY LLC        BHC

[242 rows x 4 columns]
Number of BHC rows: 242


Rows classified as Call_Parent:
     source       id                       lender_TopName   Match_Type
118     3.0  1493319           

# Summary of Part 2

Out of 413 banks:

- **242 banks** can be matched with BHC:
  - These banks have LOC IDs that directly match with the BHC IDs in the `fr_report` data.
- **15 banks** can be matched with the parent ID in the call report:
  - These banks are identified as parent banks where the `id` matches the `parent_id` in the `call_df`.
- **3 banks** can be matched as a Call_Subsidiary:
  - These banks are identified as subsidiaries where the `id` does not match the `parent_id` in the `call_df`.
  - Detailed search results for these subsidiaries:
    - **Subsidiary ID 2980209**:
      - Not found in FR report.
      - Found parent ID 5006575 in Call report for current ID 2980209.
      - Parent ID 5006575 found in FR report after searching Call report.
    - **Subsidiary ID 3614958**:
      - Not found in FR report.
      - Found parent ID 5116344 in Call report for current ID 3614958.
      - ID 5116344 found in FR report after searching Call report.
    - **Subsidiary ID 494261**:
      - Not found in FR report.
      - Found parent ID 2875332 in Call report for current ID 494261.
      - ID 2875332 found in FR report after searching Call report.
- **153 lenders** are not matched

# Part 3: For non-banks(153 lenders that are not matched), match with credit union

In [14]:
# Load your data here
cu_df = pd.read_excel('/Users/twylazhang/Desktop/Econ_banks_Research/LOC Credit Union/THIS_FOICU_Names_ID_excel.xlsx')

# Drop rows with non-finite values in RSSD column
cu_df = cu_df[pd.notnull(cu_df['RSSD'])]

# Ensure RSSD column is integer
cu_df['RSSD'] = cu_df['RSSD'].astype(int)

# Keep only the necessary columns from the credit unions file
cu_df = cu_df[['RSSD', 'CU_NAME']]

# Directly merge the unmatched LOC entries with the credit unions data on id and RSSD
numeric_ids_loc_df = numeric_ids_loc_df.merge(cu_df, left_on='id', right_on='RSSD', how='left', suffixes=('', '_cu'))

# Update Match_Type and CU_NAME for matched credit unions
numeric_ids_loc_df.loc[numeric_ids_loc_df['CU_NAME'].notna(), 'Match_Type'] = 'Credit Union'

numeric_ids_loc_df


,source,id,lender_TopName,Match_Type,highest_parent_id,RSSD,CU_NAME
0,1.0,1003790,USE CREDIT UNION,Credit Union,None,1003790.0,U. S. EMPLOYEES
1,1.0,100393,FRANKENMUTH CREDIT UNION,Credit Union,None,100393.0,FRANKENMUTH
2,3.0,100571,DEPARTMENT OF LABOR FCU,Credit Union,None,100571.0,DEPT OF LABOR
3,1.0,1025608,FIRST HAWAIIAN INC,BHC,None,NaN,NaN
4,3.0,1026801,FREMONT BANCORPORATION,BHC,None,NaN,NaN
...,...,...,...,...,...,...,...
408,3.0,967699,MIDUSA CREDIT UNION,Credit Union,None,967699.0,MYUSA
409,3.0,971986,REACH FEDERAL CREDIT UNION,No Match,None,NaN,NaN
410,3.0,978471,UNITED STATES SENATE FEDERAL CREDIT UNION,Credit Union,None,978471.0,UNITED STATES SENATE
411,1.0,986177,ADVIA CREDIT UNION,Credit Union,None,986177.0,ADVIA


After matching, 81 lenders got matched, their match type is updated to credit union

In [15]:
credit_union = numeric_ids_loc_df[numeric_ids_loc_df['Match_Type'] == 'Credit Union']
credit_union

,source,id,lender_TopName,Match_Type,highest_parent_id,RSSD,CU_NAME
0,1.0,1003790,USE CREDIT UNION,Credit Union,None,1003790.0,U. S. EMPLOYEES
1,1.0,100393,FRANKENMUTH CREDIT UNION,Credit Union,None,100393.0,FRANKENMUTH
2,3.0,100571,DEPARTMENT OF LABOR FCU,Credit Union,None,100571.0,DEPT OF LABOR
75,3.0,113777,ADVANCIAL FEDERAL CREDIT UNION,Credit Union,None,113777.0,ADVANCIAL
106,1.0,129787,FORT LEE FEDERAL CREDIT UNION,Credit Union,None,129787.0,HOMEBASE
...,...,...,...,...,...,...,...
406,3.0,948296,PARTNERSHIP FINANCIAL CREDIT UNION,Credit Union,None,948296.0,PARTNERSHIP FINANCIAL
408,3.0,967699,MIDUSA CREDIT UNION,Credit Union,None,967699.0,MYUSA
410,3.0,978471,UNITED STATES SENATE FEDERAL CREDIT UNION,Credit Union,None,978471.0,UNITED STATES SENATE
411,1.0,986177,ADVIA CREDIT UNION,Credit Union,None,986177.0,ADVIA


After matching, 72 lenders are left unmatched

In [16]:
non_matched = numeric_ids_loc_df[numeric_ids_loc_df['Match_Type'] == 'No Match']
non_matched

,source,id,lender_TopName,Match_Type,highest_parent_id,RSSD,CU_NAME
28,1.0,1070804,FIRSTMERIT CORP,No Match,None,NaN,NaN
38,1.0,1084511,COMMUNITY BK OF S FL,No Match,None,NaN,NaN
96,2.0,1231333,BANK OF MONTREAL,No Match,None,NaN,NaN
97,1.0,1231968,BNP PARIBAS,No Match,None,NaN,NaN
98,1.0,1232497,ROYAL BANK OF CANADA,No Match,None,NaN,NaN
...,...,...,...,...,...,...,...
367,1.0,5331286,PLAINS COMMERCE FINANCIAL INC,No Match,None,NaN,NaN
390,1.0,73376,FDIC FCU,No Match,None,NaN,NaN
392,1.0,742793,COMMUNITY DRIVEN CREDIT UNION,No Match,None,NaN,NaN
395,3.0,802370,ENERGY FEDERAL CREDIT UNION,No Match,None,NaN,NaN


# Summary of Part 3

Out of 153 lenders:

- **81 lenders** can be matched with Credit Union.
- **72 lenders** are left unmatched.

In [17]:
#non_matched.to_csv('non_matched.csv', index=False)

In [18]:
#credit_union.to_csv('credit_union.csv', index=False)

In [19]:
#numeric_ids_loc_df.to_csv('LOC_After_Matching.csv', index=False)

# Check
1. Check in call_df: Filter the call_df DataFrame to find rows where the 'id' matches the current LOC ID. If matching rows are found, print the details using the display_details function.

2. Check in fr_report if BHC: If the Match Type is 'BHC', filter the fr_report DataFrame to find rows where the 'id' matches the current LOC ID. If matching rows are found, print the details using the display_details function.

In [20]:
sampled_data = numeric_ids_loc_df.sample(10, random_state=1)

# Print the sampled LOC data for manual verification
print("Sampled LOC Data for Manual Verification:")
sampled_data

Sampled LOC Data for Manual Verification:


,source,id,lender_TopName,Match_Type,highest_parent_id,RSSD,CU_NAME
201,3.0,306681,MARKET USA FEDERAL CREDIT UNION,Credit Union,None,306681.0,MARKET USA
29,1.0,1073757,BANK OF AMERICA CORPORATION,BHC,None,NaN,NaN
102,1.0,1245705,WEST SUBURBAN BANCORP INC,BHC,None,NaN,NaN
407,1.0,95051,ONE AMERICAN BANK,Call_Parent,None,NaN,NaN
186,1.0,2833891,EMPRESAS JUAN YARUR SPA,No Match,None,NaN,NaN
222,1.0,3253825,LAKEVIEW BANCORPORATION INC,BHC,None,NaN,NaN
242,1.0,3470154,US METRO BK,Call_Parent,None,NaN,NaN
291,1.0,3846405,FIRST FEDERAL BANCORP MHC,BHC,None,NaN,NaN
171,1.0,2630746,UNITY BANCSHARES LLC,BHC,None,NaN,NaN
213,3.0,3188860,CALWEST BC,BHC,None,NaN,NaN


In [21]:
# Define columns to display
columns_to_display = ['id', 'name_legal', 'parent_id']

# Function to display details from a DataFrame
def display_details(df, columns):
    if not df.empty:
        print(df[columns].to_string(index=False))
    else:
        print("No details found.")

# Manual verification against call_df and fr_report
for index, row in sampled_data.iterrows():
    print(f"\nReviewing LOC ID: {row['id']} - Match Type: {row['Match_Type']}")

    # Checking in call_df
    call_details = call_df.loc[call_df['id'] == row['id'], ['id', 'name_legal', 'parent_id', 'Entity_Type']]
    if not call_details.empty:
        print("Details from call_df:")
        display_details(call_details, ['id', 'name_legal', 'parent_id', 'Entity_Type'])

    # Checking in fr_report if BHC
    if row['Match_Type'] == 'BHC':
        bhc_details = fr_report.loc[fr_report['id'] == row['id'], columns_to_display]
        if not bhc_details.empty:
            print("Details from fr_report (BHC):")
            display_details(bhc_details, columns_to_display)



Reviewing LOC ID: 306681 - Match Type: Credit Union

Reviewing LOC ID: 1073757 - Match Type: BHC
Details from fr_report (BHC):
     id                  name_legal  parent_id
1073757 BANK OF AMERICA CORPORATION          0

Reviewing LOC ID: 1245705 - Match Type: BHC
Details from fr_report (BHC):
     id                  name_legal  parent_id
1245705 WEST SUBURBAN BANCORP, INC.          0

Reviewing LOC ID: 95051 - Match Type: Call_Parent
Details from call_df:
   id        name_legal  parent_id Entity_Type
95051 ONE AMERICAN BANK      95051      Parent

Reviewing LOC ID: 2833891 - Match Type: No Match

Reviewing LOC ID: 3253825 - Match Type: BHC
Details from fr_report (BHC):
     id                    name_legal  parent_id
3253825 LAKEVIEW BANCORPORATION, INC.          0

Reviewing LOC ID: 3470154 - Match Type: Call_Parent
Details from call_df:
     id    name_legal  parent_id Entity_Type
3470154 US METRO BANK    3470154      Parent

Reviewing LOC ID: 3846405 - Match Type: BHC
Details f

 # Part 4: Link the banks in the data to their stock market identifiers

### Preprocess Data

1. **Load Data**:
   - Load the data from the URL into the `public_traded` DataFrame.

2. **Convert IDs to String**:
   - Convert the `id` and `highest_parent_id` columns in `numeric_ids_loc_df` to strings to ensure consistency.
   - Convert the `entity` column in `public_traded` to strings.

3. **Remove NaN Values**:
   - Remove NaN values in the `entity` column in the `public_traded` DataFrame.

4. **Fill `highest_parent_id` with `id` if it is empty**:
   - After filling, for entries with the match type `Call_Subsidiary`, the `highest_parent_id` is the same as before.
   - For all other match types, the `highest_parent_id` is set to the `id` itself.
   - This ensures consistency and enables straightforward matching with `public_traded` data.

### Match and Analyze Data

5. **Merge `numeric_ids_loc_df` with `public_traded` data based on the match between `highest_parent_id` in `numeric_ids_loc_df` and `entity` in `public_traded`**.

6. **Count Match Types**:
   - For those that are matched with the `public_traded` DataFrame, count their number of match types.

## Preprocess Data

In [22]:
# Load the CSV file from the URL
frb_crsp_file = "https://www.newyorkfed.org/medialibrary/media/research/banking_research/data/crsp_20161231.csv?la=en"
public_traded = pd.read_csv(frb_crsp_file, dtype=str)  # Ensure strings are not converted to factors

# Convert relevant columns to string to ensure consistency
numeric_ids_loc_df['id'] = numeric_ids_loc_df['id'].astype(str)
numeric_ids_loc_df['highest_parent_id'] = numeric_ids_loc_df['highest_parent_id'].astype(str)
public_traded['entity'] = public_traded['entity'].astype(str)

# Remove NaN values and 'nan' strings in public_traded
public_traded = public_traded.dropna(subset=['entity'])
public_traded = public_traded[public_traded['entity'] != 'nan']

# Display before filling
print("Before filling highest_parent_id:")
print(numeric_ids_loc_df[numeric_ids_loc_df['Match_Type'] == "Call_Subsidiary"][['id', 'lender_TopName','Match_Type', 'highest_parent_id']])

# Fill highest_parent_id with id if it is empty
numeric_ids_loc_df['highest_parent_id'] = numeric_ids_loc_df['highest_parent_id'].where(numeric_ids_loc_df['highest_parent_id'] != 'None', numeric_ids_loc_df['id'])

# Display after filling
print("\nAfter filling highest_parent_id:")
print(numeric_ids_loc_df[numeric_ids_loc_df['Match_Type'] == "Call_Subsidiary"][['id', 'lender_TopName','Match_Type', 'highest_parent_id']])

Before filling highest_parent_id:
          id               lender_TopName       Match_Type highest_parent_id
195  2980209       BARCLAYS BANK DELAWARE  Call_Subsidiary           5006575
260  3614958  CALIFORNIA BANK OF COMMERCE  Call_Subsidiary           5116344
354   494261         PACIFIC WESTERN BANK  Call_Subsidiary           2875332

After filling highest_parent_id:
          id               lender_TopName       Match_Type highest_parent_id
195  2980209       BARCLAYS BANK DELAWARE  Call_Subsidiary           5006575
260  3614958  CALIFORNIA BANK OF COMMERCE  Call_Subsidiary           5116344
354   494261         PACIFIC WESTERN BANK  Call_Subsidiary           2875332


highest_parent_id for Call_Subsidiary remains the same

In [23]:
# after filling
numeric_ids_loc_df

,source,id,lender_TopName,Match_Type,highest_parent_id,RSSD,CU_NAME
0,1.0,1003790,USE CREDIT UNION,Credit Union,1003790,1003790.0,U. S. EMPLOYEES
1,1.0,100393,FRANKENMUTH CREDIT UNION,Credit Union,100393,100393.0,FRANKENMUTH
2,3.0,100571,DEPARTMENT OF LABOR FCU,Credit Union,100571,100571.0,DEPT OF LABOR
3,1.0,1025608,FIRST HAWAIIAN INC,BHC,1025608,NaN,NaN
4,3.0,1026801,FREMONT BANCORPORATION,BHC,1026801,NaN,NaN
...,...,...,...,...,...,...,...
408,3.0,967699,MIDUSA CREDIT UNION,Credit Union,967699,967699.0,MYUSA
409,3.0,971986,REACH FEDERAL CREDIT UNION,No Match,971986,NaN,NaN
410,3.0,978471,UNITED STATES SENATE FEDERAL CREDIT UNION,Credit Union,978471,978471.0,UNITED STATES SENATE
411,1.0,986177,ADVIA CREDIT UNION,Credit Union,986177,986177.0,ADVIA


## Match and Analyze Data

### Create LOC_Public_Traded_Merged and count match type

In [24]:
# Merge based on 'highest_parent_id' only
LOC_Public_Traded_Merged = pd.merge(numeric_ids_loc_df, public_traded, left_on='highest_parent_id', right_on='entity', how='left', suffixes=('', '_public_traded'))

# Count the match types and their corresponding number of match types for merged rows
match_type_counts_merged = LOC_Public_Traded_Merged[~LOC_Public_Traded_Merged['entity'].isna()]['Match_Type'].value_counts()

print("Match type counts in LOC_Public_Traded_Merged (only matched with public_traded):")
print(match_type_counts_merged)

Match type counts in LOC_Public_Traded_Merged (only matched with public_traded):
Match_Type
BHC                117
No Match            13
Call_Parent          1
Call_Subsidiary      1
Name: count, dtype: int64


In [25]:
matched_rows = LOC_Public_Traded_Merged[~LOC_Public_Traded_Merged['entity'].isna()]
matched_rows

,source,id,lender_TopName,Match_Type,highest_parent_id,RSSD,CU_NAME,notice,name,entity,permco,dt_start,dt_end
3,1.0,1025608,FIRST HAWAIIAN INC,BHC,1025608,NaN,NaN,NaN,Bancwest Corporation,1025608,1718,19860630,20011231
4,1.0,1025608,FIRST HAWAIIAN INC,BHC,1025608,NaN,NaN,NaN,"First Hawaiian, Inc.",1025608,55670,20160804,20161231
6,3.0,1027004,ZIONS BC,BHC,1027004,NaN,NaN,NaN,Zions Bancorporation,1027004,5057,19860630,20161231
7,1.0,1030040,BANK OF CMRC HOLD,BHC,1030040,NaN,NaN,NaN,Bank Of Commerce Holdings,1030040,45295,20040630,20161231
8,1.0,1030170,TRICO BANCSHARES,BHC,1030170,NaN,NaN,NaN,Trico Bancshares,1030170,12086,19930430,20161231
...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,1.0,4284536,CUSTOMERS BANCORP INC,BHC,4284536,NaN,NaN,NaN,Customers Bancorp Inc,4284536,54408,20130531,20161231
335,1.0,4366003,HOMETRUST BANCSHARES INC,BHC,4366003,NaN,NaN,NaN,"Hometrust Bancshares, Inc.",4366003,54151,20120731,20161231
343,3.0,4523431,WATERSTONE FINANCIAL INC,BHC,4523431,NaN,NaN,NaN,Waterstone Financial Inc,4523431,47887,20140123,20161231
351,3.0,4809920,GREAT WESTERN BANCORP INC,BHC,4809920,NaN,NaN,NaN,Great Western Bancorp Inc,4809920,55050,20141031,20161231


# Summary of Part 4

Out of **413** banks:

Only **132** banks can be matched with the `entity` column from the `public_traded` DataFrame, meaning only 132 banks are publicly traded.

Match type counts in these 132 banks:

- **BHC**: 117
- **No Match**: 13
- **Call_Parent**: 1
- **Call_Subsidiary**: 1

In [26]:
#LOC_Public_Traded_Merged.to_csv('LOC_Public_Traded_Merged.csv', index=False)

In [27]:
#matched_rows.to_csv('LOC_Public_Traded_Merged_Matched_Rows.csv', index=False)

# a function in python (based on chatgpt) that allows you to keep non-merged rows. 

In [28]:
# import pandas as pd

# # Example data for df1
# data1 = {
#     'common_id': [1, 2, 3, 4],
#     'value_df1': ['A', 'B', 'C', 'D']
# }
# df1 = pd.DataFrame(data1)

# # Example data for df2
# data2 = {
#     'common_id': [3, 4, 5, 6],
#     'value_df2': ['X', 'Y', 'Z', 'W']
# }
# df2 = pd.DataFrame(data2)

# # Merge the DataFrames
# merged_df = pd.merge(df1, df2, on='common_id', how='outer')

# # Add a new column to indicate matching status
# merged_df['match_status'] = 'matched'
# merged_df.loc[merged_df['value_df1'].isna() | merged_df['value_df2'].isna(), 'match_status'] = 'notmatched'

# # Optionally, fill missing values
# merged_df.fillna('not available', inplace=True)

# # Display the resulting DataFrame
# print(merged_df)